In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

## Merge mortality et worldbank

### Modifications sur df_mortality

In [2]:
df_mort= pd.read_table('mortality.txt',delimiter=';',names=['Cancer_code', 'Sex', 'Age_bucket', 'Location', 'Year', 'Nb_deaths'],low_memory=False)
df_mort.describe()

,Sex,Age_bucket,Year,Nb_deaths
count,6.359642e+06,6.359642e+06,6.359642e+06,6.359642e+06
mean,1.540061e+00,1.000000e+01,1.992326e+03,3.230369e+01
std,6.528709e-01,5.477226e+00,1.609652e+01,2.649799e+02
min,1.000000e+00,1.000000e+00,1.950000e+03,0.000000e+00
25%,1.000000e+00,5.000000e+00,1.981000e+03,0.000000e+00
50%,2.000000e+00,1.000000e+01,1.996000e+03,0.000000e+00
75%,2.000000e+00,1.500000e+01,2.006000e+03,5.000000e+00
max,9.000000e+00,1.900000e+01,2.015000e+03,3.493500e+04


In [ ]:
df_mort.tail(5)

- Aggregation de la tranche d'age 16 et suppression des tranches d'age 17,18 et 19

In [3]:
length=len(df_mort[df_mort['Age_bucket']==16])
index=df_mort[df_mort['Age_bucket']==16]['Nb_deaths'].index
j=list(range(0, length, 1))
t_16=df_mort[df_mort['Age_bucket']==16]['Nb_deaths'].values
t_17=df_mort[df_mort['Age_bucket']==17]['Nb_deaths'].values
t_18=df_mort[df_mort['Age_bucket']==18]['Nb_deaths'].values
t_19=df_mort[df_mort['Age_bucket']==19]['Nb_deaths'].values
df_mort['Nb_deaths'][index]=t_16[j] + t_17[j] + t_18[j] + t_19[j]

C:\Users\DIARRA née CISSE Bal\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
index17_18_19=df_mort[(df_mort['Age_bucket']==17) | (df_mort['Age_bucket']==18) | (df_mort['Age_bucket']==19)]['Nb_deaths'].index
df_mort=df_mort.drop(df_mort.index[index17_18_19])
#df_mort.head(19)

In [5]:
#Attention il faut modifier la derniere ligne
df_age = pd.read_csv('age.csv', header = None , sep=' ')
df_age.columns=['Age_bucket', 'Age']
df_age.head()

,Age_bucket,Age
0,1,0-4
1,2,5-9
2,3,10-14
3,4,15-19
4,5,20-24


In [6]:
#read file - cancer codes
df_cancer = pd.read_csv('cancer_codes.csv', header=None, sep='\t')
df_cancer.columns = ['Cancer_code', 'Cancer_label']
df_cancer.head()

,Cancer_code,Cancer_label
0,C00-C14,"Malignant neoplasms of lip, oral cavity and ph..."
1,C15,Malignant neoplasm of oesophagus
2,C15-C26,Malignant neoplasm of digestive organs and per...
3,C16,Malignant neoplasm of stomach
4,C17,"Malignant neoplasm of small intestine, includi..."


In [7]:
#merge age class df + main df
df1 = pd.merge(df_mort, df_age, how='left', left_on='Age_bucket', right_on='Age_bucket')
df1.head()

,Cancer_code,Sex,Age_bucket,Location,Year,Nb_deaths,Age
0,C80,2,1,Jamaica,2000,2,0-4
1,C80,2,2,Jamaica,2000,0,5-9
2,C80,2,3,Jamaica,2000,1,10-14
3,C80,2,4,Jamaica,2000,0,15-19
4,C80,2,5,Jamaica,2000,0,20-24


- Harmonisation des noms des pays entre les deux fichiers "mortality" et "worldbank"

In [8]:
#read csv file - WorldBank indicators
df_wb = pd.read_csv('WorldBank_Data.csv', sep=',')

In [9]:
df_correspondance=pd.read_table('correspondance.txt',delimiter=';')
df_correspondance.head()
#df_correspondance.describe()

,area,area2
0,Afghanistan,
1,Albania,Albania
2,Algeria,
3,American Samoa,
4,Andorra,


In [10]:
Freq_area2 = df_correspondance.groupby(['area2'])['area2'].count()

In [11]:
# on crée une nouvelle colonne "area2" dane la base worldbank qui correspond au nom du pays dans la base mortality
df_wb2 = pd.merge(df_wb, df_correspondance, on='area')
df_wb2['area'].describe() #258 pays différents
df_wb2['area2'].describe() #131 pays différents

count     11648
unique      131
top            
freq       5663
Name: area2, dtype: object

Ensuite, il faudra faire le merge entre la table mortality et worldbank avec la variable area2 au lieu de area. Dans la table mortality, on a 153 pays. Dans la table worldbank, on a 258 pays. En fesant, un merge join='inner' entre les 2 bases, on aura plus que 130 pays différents. En effet, on perd 23 pays de la base mortality. Ce sont pour la plupart des iles, des pays qui n'exitent plus, ou encore des regroupements de pays qui ne sont pas présents dans la base worldbank.

In [13]:
#calculate new variables : transform percentage into absolute values for each group
#naming convention = old name without the .5Y at the end and without the SP in the beginning
pop_par_tranche_dage=['POP.0004.','POP.0509.','POP.1014.','POP.1519.','POP.2024.','POP.2529.','POP.3034.','POP.3539.','POP.4044.','POP.4549.','POP.5054.','POP.5559.','POP.6064.','POP.6569.','POP.7074.','POP.7579.','POP.80UP.']
for i in range(0,17):
    df_wb2[''+pop_par_tranche_dage[i]+'FE'] =(df_wb2['SP.POP.TOTL']*df_wb2['SP.POP.TOTL.FE.ZS']*df_wb2['SP.'+pop_par_tranche_dage[i]+'FE.5Y'])/10000
    df_wb2[''+pop_par_tranche_dage[i]+'MA'] =(df_wb2['SP.POP.TOTL']*(100-df_wb2['SP.POP.TOTL.FE.ZS'])*df_wb2['SP.'+pop_par_tranche_dage[i]+'MA.5Y'])/10000


In [14]:
## subsetting dataframe with columns to merge
df_mini = df_wb2[['area','year','POP.0004.FE', 'POP.0509.FE', 'POP.1014.FE', 'POP.1519.FE', 'POP.2024.FE', 'POP.2529.FE', 'POP.3034.FE', 'POP.3539.FE', 'POP.4044.FE', 'POP.4549.FE', 'POP.5054.FE', 'POP.5559.FE', 'POP.6064.FE', 'POP.6569.FE', 'POP.7074.FE', 'POP.7579.FE', 'POP.80UP.FE', 'POP.0004.MA', 'POP.0509.MA', 'POP.1014.MA', 'POP.1519.MA', 'POP.2024.MA', 'POP.2529.MA', 'POP.3034.MA', 'POP.3539.MA', 'POP.4044.MA', 'POP.4549.MA', 'POP.5054.MA', 'POP.5559.MA', 'POP.6064.MA', 'POP.6569.MA', 'POP.7074.MA', 'POP.7579.MA', 'POP.80UP.MA']]
df_mini.head()

,area,year,POP.0004.FE,POP.0509.FE,POP.1014.FE,POP.1519.FE,POP.2024.FE,POP.2529.FE,POP.3034.FE,POP.3539.FE,...,POP.3539.MA,POP.4044.MA,POP.4549.MA,POP.5054.MA,POP.5559.MA,POP.6064.MA,POP.6569.MA,POP.7074.MA,POP.7579.MA,POP.80UP.MA
0,Aruba,1970,3434.0,3793.0,3586.0,3365.0,2879.0,2302.0,1946.0,1915.0,...,1544.0,1316.0,1175.0,1056.0,943.0,886.0,524.0,295.0,144.0,76.0
1,Aruba,1971,3218.0,3713.0,3578.0,3365.0,2975.0,2415.0,2003.0,1955.0,...,1602.0,1348.0,1172.0,1051.0,930.0,895.0,579.0,320.0,160.0,85.0
2,Aruba,1972,3109.0,3574.0,3580.0,3343.0,3043.0,2531.0,2076.0,1973.0,...,1655.0,1393.0,1174.0,1044.0,924.0,881.0,640.0,345.0,178.0,95.0
3,Aruba,1973,3075.0,3374.0,3585.0,3311.0,3078.0,2648.0,2160.0,1980.0,...,1704.0,1449.0,1188.0,1033.0,922.0,854.0,700.0,371.0,197.0,107.0
4,Aruba,1974,3045.0,3143.0,3563.0,3289.0,3082.0,2754.0,2250.0,2000.0,...,1755.0,1512.0,1214.0,1025.0,921.0,827.0,743.0,405.0,216.0,121.0


In [15]:
#list of variable names about sex= female / male
df_mini.columns[df_mini.columns.str.endswith('FE')]
df_mini.columns[df_mini.columns.str.endswith('MA')]

Index(['POP.0004.MA', 'POP.0509.MA', 'POP.1014.MA', 'POP.1519.MA',
       'POP.2024.MA', 'POP.2529.MA', 'POP.3034.MA', 'POP.3539.MA',
       'POP.4044.MA', 'POP.4549.MA', 'POP.5054.MA', 'POP.5559.MA',
       'POP.6064.MA', 'POP.6569.MA', 'POP.7074.MA', 'POP.7579.MA',
       'POP.80UP.MA'],
      dtype='object')

In [16]:
#reshaping
df_mini.set_index(['area', 'year'], inplace=True)
df_mini.columns = pd.MultiIndex.from_tuples(tuple(df_mini.columns.str.split(".")))

transf = df_mini.stack(level = [1,2]).reset_index()
transf.columns = ['Country', 'Year', 'Age_group', 'Sex', 'Population']

In [17]:
#match labels
'''
MA/FE -> 1/2
0004/0509/1014... -> 1/2/3...
!!!classes in worldbank folder cap at 80, mortality folder has more classes(17:80-84, 18:85+, 19:unknown)
'''
transf['Sex'].replace(['MA', 'FE'],[1,2], inplace=True)
transf['Age_group'].replace(['0004','0509','1014','1519','2024','2529','3034','3539','4044','4549','5054','5559','6064','6569','7074','7579'],
list(range(1, 17)), inplace=True)

In [18]:
#merge mortality data with population transformed dataframe
#aggreger la tranche d'age
df = pd.merge(df_mort, 
              transf, 
              how='inner', 
              left_on=['Location', 'Year', 'Age_bucket', 'Sex'], 
right_on=['Country', 'Year', 'Age_group', 'Sex'])
df.head(5)

,Cancer_code,Sex,Age_bucket,Location,Year,Nb_deaths,Country,Age_group,Population
0,C80,2,1,Jamaica,2000,2,Jamaica,1,139549.999999
1,C56,2,1,Jamaica,2000,0,Jamaica,1,139549.999999
2,C64,2,1,Jamaica,2000,1,Jamaica,1,139549.999999
3,C44,2,1,Jamaica,2000,0,Jamaica,1,139549.999999
4,C84,2,1,Jamaica,2000,0,Jamaica,1,139549.999999


In [19]:
#calculate mortality as a rate
df['Mortality_rate'] = df['Nb_deaths'] / df['Population']
df['Mortality_rate'].describe()

count    3.652496e+06
mean     1.098289e-04
std      6.163104e-04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.171180e-05
max      8.091666e-02
Name: Mortality_rate, dtype: float64

In [20]:
df.head(3)

,Cancer_code,Sex,Age_bucket,Location,Year,Nb_deaths,Country,Age_group,Population,Mortality_rate
0,C80,2,1,Jamaica,2000,2,Jamaica,1,139549.999999,0.000014
1,C56,2,1,Jamaica,2000,0,Jamaica,1,139549.999999,0.000000
2,C64,2,1,Jamaica,2000,1,Jamaica,1,139549.999999,0.000007


In [21]:
#Les taux ne sont pas du tout significatifs, on decide de multiplier par 100000
df['Mortality_rate']=df['Mortality_rate']*100000

### Merge avec wordbank pour récupérer les indicateurs

In [22]:
indicators = ['area','year','SP.URB.TOTL.IN.ZS',	'SP.URB.GROW',	'SP.POP.GROW',	'SE.PRM.DURS',	'AG.SRF.TOTL.K2',	'SE.PRE.DURS',	'EN.POP.DNST',	'SE.SEC.DURS',	'SP.DYN.CBRT.IN',	'SP.DYN.CDRT.IN',	'NY.ADJ.AEDU.GN.ZS',	'SP.DYN.TFRT.IN',	'SP.ADO.TFRT',	'SP.DYN.LE00.FE.IN',	'SP.DYN.LE00.IN',	'SP.POP.DPND',	'SP.DYN.AMRT.FE',	'SP.DYN.AMRT.MA',	'EN.ATM.NOXE.KT.CE',	'EN.ATM.METH.KT.CE',	'AG.LND.AGRI.K2',	'EN.ATM.GHGO.KT.CE',	'AG.LND.AGRI.ZS',	'AG.PRD.FOOD.XD',	'IT.CEL.SETS',	'AG.PRD.LVSK.XD',	'AG.PRD.CROP.XD',	'IT.MLT.MAIN',	'EN.ATM.GHGT.KT.CE',	'EN.ATM.CO2E.GF.ZS',	'EN.ATM.CO2E.LF.ZS',	'EN.ATM.CO2E.SF.ZS',	'NY.GDP.FRST.RT.ZS',	'NY.GDP.MINR.RT.ZS',	'EN.ATM.CO2E.LF.KT',	'EN.ATM.METH.AG.KT.CE',	'EN.ATM.METH.EG.KT.CE',	'EN.ATM.NOXE.AG.KT.CE',	'EN.ATM.NOXE.EG.KT.CE',	'NY.GDP.TOTL.RT.ZS',	'EN.ATM.CO2E.KT',	'EN.ATM.CO2E.PC',	'NY.GDP.MKTP.CD',	'NY.GDP.PCAP.CD']
df2=pd.merge(df, 
              df_wb2.loc[:,indicators], 
              how='inner',
              left_on=['Location','Year'], 
         right_on=['area','year'])

In [23]:
df2.isnull().values.sum()

8454928

## Nettoyage du dataframe

### On remarque deux choses:
- Il y a des pays comme Uzbekistan qui fonctionnent mieux avec bfill
- Il y a des pays comme l'Anguilla qui fonctionnent mieux avec ffill
On decide donc de faire un ffill puis un bfill

In [24]:
df2=df2.groupby('Country').apply(lambda x: (x.sort_values('Year', ascending=True)))
df2.fillna(method='ffill',inplace=True)
df2.fillna(method='bfill',inplace=True)
df2.reset_index(drop=True,inplace=True)
df2.isnull().values.sum()

0

On se restreint au cancer du cerveau à partir de l'an 2000

In [26]:
df3=df2[(df2['Year'] >= 2000) & (df2['Cancer_code']=='C33,C34') ]

In [27]:
df3.drop(['Cancer_code','Location','Nb_deaths','Age_group','Population','area','year'],inplace=True,axis=1)

C:\Users\DIARRA née CISSE Bal\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
df3.describe(include='all') 
#qcut:quantile
#dummy regressor dummy classifier

,Sex,Age_bucket,Year,Country,Mortality_rate,SP.URB.TOTL.IN.ZS,SP.URB.GROW,SP.POP.GROW,SE.PRM.DURS,AG.SRF.TOTL.K2,...,EN.ATM.METH.AG.KT.CE,EN.ATM.METH.EG.KT.CE,EN.ATM.NOXE.AG.KT.CE,EN.ATM.NOXE.EG.KT.CE,NY.GDP.TOTL.RT.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,Country_code
count,37264.000000,37264.000000,37264.000000,37264,37264.000000,37264.000000,37264.000000,37264.000000,37264.000000,3.726400e+04,...,37264.000000,37264.000000,37264.000000,37264.000000,37264.000000,3.726400e+04,37264.000000,3.726400e+04,37264.000000,37264.000000
unique,NaN,NaN,NaN,99,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,Hungary,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,512,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.499785,8.500000,2007.176900,NaN,50.884518,67.660807,1.252047,0.941133,5.598111,7.703405e+05,...,13970.297149,11629.266864,8477.428963,1013.843177,5.228333,1.327179e+05,7.527603,3.562845e+11,18949.875301,48.231430
std,0.500007,4.609834,4.243968,NaN,132.585822,18.536914,1.801041,1.599175,0.978053,2.351910e+06,...,36415.347075,39544.807912,19171.654263,1748.459431,10.057098,2.527069e+05,7.618949,7.982917e+11,20487.233315,28.675676
min,1.000000,1.000000,2000.000000,NaN,0.000000,9.092000,-2.697859,-2.850973,3.000000,1.800000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.566900e+01,0.180554,6.310127e+07,329.781984,0.000000
25%,1.000000,4.750000,2004.000000,NaN,0.000000,54.526000,0.256457,0.181115,5.000000,2.207000e+04,...,613.661061,367.499687,332.796577,58.667239,0.188408,7.425675e+03,2.723018,1.437327e+10,4146.987928,23.000000
50%,1.000000,8.500000,2007.000000,NaN,1.260432,68.125000,1.045134,0.745507,6.000000,9.303000e+04,...,3221.288138,1763.285705,2390.035148,201.079454,1.192305,4.294424e+04,5.463501,5.713624e+10,10353.447983,48.000000
75%,2.000000,12.250000,2011.000000,NaN,32.844219,82.220000,1.908281,1.424580,6.000000,3.851780e+05,...,11915.265350,11734.036958,8063.684071,807.709166,4.784801,1.069077e+05,9.318429,2.699801e+11,27726.481054,73.000000


In [28]:
import tempfile
tempfile.tempdir='C:/Temp/'
with tempfile.NamedTemporaryFile() as temp:

    df3.to_csv(temp.name + '.csv')